In [ ]:
from pypdf import PdfReader
from pathlib import Path
import wordninja
import csv
import re

bini = Path("/home/kinged/codes/dala/group_a/bini_dict.pdf")


the bini dictionary starts from page 18 down to page 251 for all the word and translations used to form the coropra, the latter pages may contain some other useful information that can be used in some manner to help in translations and effective corpora development 

In [10]:
page = r"/home/kinged/codes/dala/group_a/bini_dict.pdf"
path = r"/home/kinged/codes/dala/group_a/text"

# function to extract the pages from the pdfs

def extract_texts(file_path: str, dest_path: str):
    """
    **extract the pages from the pdf and store them as text files inside a specified directory**
    
    Args:
        file_path(str): path to pdf file
        dest_path(str): path to the destination folder

    Returns:
        None
    """

    page = PdfReader(Path(file_path))
    path = Path(dest_path)
    if path.exists() == True:
        pass
    else:
        path.mkdir()

    for x in range(18, page.get_num_pages()):
        with open(f"{path}/pages{x}.txt", "w") as f:
            f.write(page.pages[x].extract_text())

extract_texts(page, path)


In [11]:
# function for transforming the files

texts = Path("/home/kinged/codes/dala/group_a/text")
cleaned_dir = Path("/home/kinged/codes/dala/group_a/cleaned_text")
cleaned_dir.mkdir(exist_ok=True)

def clean_text(text):
    # Remove special annotation symbols (e.g., [.*], [..], etc.)
    text = re.sub(r'\[[^\]]*\]', '', text)
    # Replace line breaks within words with a space
    text = re.sub(r'([a-zA-Z0-9])\n([a-zA-Z0-9])', r'\1 \2', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)
    # Attempt to split merged words using wordninja
    words = []
    for token in text.split():
        if len(token) > 15:  # likely merged
            words.extend(wordninja.split(token))
        else:
            words.append(token)
    return ' '.join(words)

def clean_texts_in_folder(folder_path, output_folder):
    for fpath in folder_path.glob('*.txt'):
        with open(fpath, 'r', encoding='utf-8') as f:
            raw = f.read()
        cleaned = clean_text(raw)
        out_path = output_folder / fpath.name.replace('.txt', '_cleaned.txt')
        with open(out_path, 'w', encoding='utf-8') as f:
            f.write(cleaned)

# Run cleaning on all text files in the folder
clean_texts_in_folder(texts, cleaned_dir)



In [12]:
import pandas as pd

def create_corpus_csv(cleaned_folder, output_csv):
    corpus = []
    for fpath in Path(cleaned_folder).glob('*_cleaned.txt'):
        with open(fpath, 'r', encoding='utf-8') as f:
            text = f.read()
            # Split entries by semicolon or newline, strip whitespace
            entries = [entry.strip() for entry in re.split(r'[;\n]', text) if entry.strip()]
            for entry in entries:
                corpus.append({'text': entry, 'source_file': fpath.name})
    df = pd.DataFrame(corpus)
    df.to_csv(output_csv, index=False)

# Create corpus CSV from cleaned data
create_corpus_csv('/home/kinged/codes/dala/group_a/cleaned_text', '/home/kinged/codes/dala/group_a/corpus.csv')

ModuleNotFoundError: No module named 'pandas'